## baidu 翻译 API

In [5]:
import requests
import random
import json
from hashlib import md5


def en2zh(query):
    query = query.strip()
    if query is None or len(query) == 0:
        return ""
    
    # Set your own appid/appkey.
    appid = '20220720001277966'
    appkey = 'nN9pUmkp8Sm8skkanHMu'

    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang =  'zh'

    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    # Generate salt and sign
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()

    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}

    try:
        # Send request
        r = requests.post(url, params=payload, headers=headers)
        res = r.json()
        trans_res = res['trans_result']
        temp = []
        for tran in trans_res:
            temp.append(tran['dst'])
        final_res = "\n".join(temp)
    except Exception as e:
        print(e)
        final_res = "Error Translate"
    return final_res

In [6]:
result = en2zh("Rewrite the sentence into more descriptive form:\nThe food was delicious.")

'trans_result'


In [7]:
result

'Error Translate'

## Helsinki-NLP/opus-mt-en-zh
英-中翻译模型

In [ ]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh", device=0)

In [39]:
import re
def en2zh_by_model(query):
    query = query.strip()
    if not query:
        return ""
    # input 中含有代码片段
    if re.search("^def.*", query):
        return query
    if re.search("^Class.*", query):
        return query
    # 当一句话中有 \n 时，需要拆开喂到模型，最后再用 \n 拼接
    q_list = query.split('\n')
    temp = []
    for q in q_list:
        temp.append(translator(q)[0]['translation_text'])
    res = "\n".join(temp)
    return res

In [46]:
en2zh_by_model("this is a test, 7x * 8 = y")

'这是一个测试, 7x * 8=y'

In [41]:
import pandas as pd
raw_data = pd.read_json('./alpaca_data.json')

In [11]:
raw_data

,instruction,input,output
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain..."
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...
...,...,...,...
51997,Generate an example of what a resume should li...,,"Jean Tremaine\n1234 Main Street, Anytown, CA 9..."
51998,Arrange the items given below in the order to ...,"cake, me, eating",I eating cake.
51999,Write an introductory paragraph about a famous...,Michelle Obama,Michelle Obama is an inspirational woman who r...
52000,Generate a list of five things one should keep...,,1. Research potential opportunities and carefu...


In [44]:
from tqdm import tqdm
tqdm.pandas(desc='translate')
temp = raw_data.instruction.progress_map(lambda x: en2zh_by_model(x))

translate: 100%|██████████| 52002/52002 [3:40:10<00:00,  3.94it/s]   


In [122]:
#zh_data.to_json('./temp/zh_alpaca_data.json', orient="records", force_ascii=False)

In [48]:
raw_data['zh_instruction'] = temp

In [49]:
raw_data

,instruction,input,output,zh_instruction
0,Give three tips for staying healthy.,,1.Eat a balanced diet and make sure to include...,给三个忠告 保持健康。
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye...",三种主要颜色是什么?
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain...",描述原子的结构。
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...,我们如何减少空气污染?
4,Describe a time when you had to make a difficu...,,I had to make a difficult decision when I was ...,描述你不得不作出困难决定的时间。
...,...,...,...,...
51997,Generate an example of what a resume should li...,,"Jean Tremaine\n1234 Main Street, Anytown, CA 9...","生成一个实例, 说明烹饪专业的 履历会想要什么 。"
51998,Arrange the items given below in the order to ...,"cake, me, eating",I eating cake.,"将下列项目排列如下,以完成以下句子:"
51999,Write an introductory paragraph about a famous...,Michelle Obama,Michelle Obama is an inspirational woman who r...,写一段关于名人的介绍性段落。
52000,Generate a list of five things one should keep...,,1. Research potential opportunities and carefu...,"在考虑职业变化时,要提出一份五件事的清单。"
